In [3]:
#Import dependencies

#Data manipulation
import pandas as pd

#Calculating goodman Kruskl Gamma
from scipy.stats import kendalltau

#Logistic Regression
import statsmodels.api as sm
import statsmodels.formula.api as smf

#Machine learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [4]:
#Reading the data
df=pd.read_csv("data.csv")

In [5]:
#Checking the first 5 rows
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [6]:
df.drop(['id','Unnamed: 32'], axis=1, inplace=True)

In [7]:
#Replace M's with 1 and B's with 0
df.diagnosis.replace({"M":1,"B":0},inplace=True)
df.diagnosis.unique()

array([1, 0], dtype=int64)

In [8]:
X=df.drop(columns='diagnosis')
Y=df['diagnosis']

In [9]:
#Calculating Goodman Kruskal Gamma
gammas=[]

In [10]:
for col in X.columns:
    tau, pvalue = kendalltau(X[col], Y)
    gamma = 2 * tau / (1 + tau)
    gammas.append(gamma)

In [11]:
print(gammas)

[0.7492820468507292, 0.5482461588914271, 0.7591322867166297, 0.7499939113137851, 0.4662963388686181, 0.6648606098351186, 0.749569514832059, 0.7774113439577374, 0.42757919323590776, -0.043262238448845344, 0.6703870189562042, 0.031248080809407247, 0.6800793686256391, 0.737151757669114, -0.08911277847895073, 0.47458053227422475, 0.5554202811957164, 0.5709648834318791, -0.16321608813809974, 0.2827811766282231, 0.7835991405068972, 0.5607929954391039, 0.7885239373904993, 0.7828008883115565, 0.5162679822265422, 0.6630636302318272, 0.7316942827224027, 0.7798110407145356, 0.48985960164351844, 0.40582894111366247]


In [12]:
threshold = 0.4
X_filtered = X.loc[:, [gamma >= threshold for gamma in gammas]]

In [13]:
print(X_filtered.columns)

Index(['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'radius_se', 'perimeter_se',
       'area_se', 'compactness_se', 'concavity_se', 'concave points_se',
       'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst',
       'smoothness_worst', 'compactness_worst', 'concavity_worst',
       'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')


In [14]:
X_filtered.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,radius_se,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,1.0950,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.5435,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.7456,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.4956,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.7572,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [15]:
#dropping the irrelevant columns such as perimeter and area will depend on radius , the worst factors are a subsets of mean factors so we might drop them as well

In [16]:
col1=['perimeter_mean', 'area_mean', 'perimeter_se',
       'area_se','concave points_mean', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst',
    'smoothness_worst', 'compactness_worst', 'concavity_worst',
       'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst', 'concave points_se']

In [17]:
X1=X_filtered.drop(col1, axis=1)

In [18]:
X1

,radius_mean,texture_mean,smoothness_mean,compactness_mean,concavity_mean,symmetry_mean,radius_se,compactness_se,concavity_se
0,17.99,10.38,0.11840,0.27760,0.30010,0.2419,1.0950,0.04904,0.05373
1,20.57,17.77,0.08474,0.07864,0.08690,0.1812,0.5435,0.01308,0.01860
2,19.69,21.25,0.10960,0.15990,0.19740,0.2069,0.7456,0.04006,0.03832
3,11.42,20.38,0.14250,0.28390,0.24140,0.2597,0.4956,0.07458,0.05661
4,20.29,14.34,0.10030,0.13280,0.19800,0.1809,0.7572,0.02461,0.05688
...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,0.11100,0.11590,0.24390,0.1726,1.1760,0.02891,0.05198
565,20.13,28.25,0.09780,0.10340,0.14400,0.1752,0.7655,0.02423,0.03950
566,16.60,28.08,0.08455,0.10230,0.09251,0.1590,0.4564,0.03731,0.04730
567,20.60,29.33,0.11780,0.27700,0.35140,0.2397,0.7260,0.06158,0.07117


In [19]:
X1['diagnosis']=Y

In [20]:
X1.head()

,radius_mean,texture_mean,smoothness_mean,compactness_mean,concavity_mean,symmetry_mean,radius_se,compactness_se,concavity_se,diagnosis
0,17.99,10.38,0.11840,0.27760,0.3001,0.2419,1.0950,0.04904,0.05373,1
1,20.57,17.77,0.08474,0.07864,0.0869,0.1812,0.5435,0.01308,0.01860,1
2,19.69,21.25,0.10960,0.15990,0.1974,0.2069,0.7456,0.04006,0.03832,1
3,11.42,20.38,0.14250,0.28390,0.2414,0.2597,0.4956,0.07458,0.05661,1
4,20.29,14.34,0.10030,0.13280,0.1980,0.1809,0.7572,0.02461,0.05688,1


In [21]:
y=X1['diagnosis']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.3)

In [23]:
cols=X1.columns.drop('diagnosis')
formula = 'diagnosis ~ ' + ' + '.join(cols)
print(formula, '\n')
print(X_train)

diagnosis ~ radius_mean + texture_mean + smoothness_mean + compactness_mean + concavity_mean + symmetry_mean + radius_se + compactness_se + concavity_se 

     radius_mean  texture_mean  smoothness_mean  compactness_mean   
59         8.618         11.79          0.09752           0.05272  \
479       16.250         19.51          0.10260           0.18930   
29        17.570         15.05          0.09847           0.11570   
226       10.440         15.46          0.10530           0.07722   
520        9.295         13.90          0.13710           0.12250   
..           ...           ...              ...               ...   
73        13.800         15.79          0.10070           0.12800   
460       17.080         27.15          0.09898           0.11100   
189       12.300         15.90          0.08080           0.07253   
485       12.450         16.41          0.09514           0.15110   
15        14.540         27.54          0.11390           0.15950   

     concavity_m

In [24]:
model = smf.glm(formula=formula, data=X_train, family=sm.families.Binomial())
logistic_fit=model.fit()

In [25]:
print(logistic_fit.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:              diagnosis   No. Observations:                  398
Model:                            GLM   Df Residuals:                      388
Model Family:                Binomial   Df Model:                            9
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -47.453
Date:                Thu, 20 Apr 2023   Deviance:                       94.906
Time:                        11:23:47   Pearson chi2:                     124.
No. Iterations:                     9   Pseudo R-squ. (CS):             0.6643
Covariance Type:            nonrobust                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          -36.4900      7.265  

In [26]:
predictions = logistic_fit.predict(X_test)

In [27]:
print(predictions)

270    0.001849
493    0.000088
218    0.999958
294    0.000361
425    0.000321
         ...   
310    0.002496
95     0.999991
169    0.085789
446    0.999998
327    0.000324
Length: 171, dtype: float64


In [28]:
predictions_binary = [ 0 if x < 0.5 else 1 for x in predictions]

In [29]:
print(predictions_binary)

[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0]


In [30]:
cm= confusion_matrix(y_test,predictions_binary)

true_negative = cm[0][0]
false_positive = cm[0][1]
false_negative = cm[1][0]
true_positive = cm[1][1]

print('Confusion Matrix: \n', cm, '\n')

print('True Negative:', true_negative)
print('False Positive:', false_positive)
print('False Negative:', false_negative)
print('True Positive:', true_positive)
print('Correct Predictions', 
      round((true_negative + true_positive) / len(predictions_binary) * 100, 1), '%')

Confusion Matrix: 
 [[109   2]
 [  2  58]] 

True Negative: 109
False Positive: 2
False Negative: 2
True Positive: 58
Correct Predictions 97.7 %
